<a href="https://colab.research.google.com/github/ajit-gvs/HCR/blob/master/hcr_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
print(tf.__version__)
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from PIL import Image
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True



In [2]:
def image_binarization(image):
  # converting image to grayscale
  image_grayscale=image.convert('L')
  img=np.array(image_grayscale) 
   
  img[img <128] = 0
  img[img >=128] = 254 

  #making white as foreground pixels and black as background pixels
  img[img==0]=255
  img[img==254]=0
  
  return img


def line_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a line
  
  lines=[]
  begin_matrix=[]
  stop_matrix=[]
  del_start_matrix=[]
  del_end_matrix=[]

  horizontal_hist = np.sum(img,axis=1,keepdims=True)/255
  start_count=0

  for i in range(len(horizontal_hist)):

    if horizontal_hist[i]>0 and horizontal_hist[i-1]==0:
      start_count+=1
      start_matrix.append(i)

    if horizontal_hist[i]==0 and start_count>0 and horizontal_hist[i-1]>0:
      end_matrix.append(i)

  
  
  if len(start_matrix)==len(end_matrix):
    for i in range(len(start_matrix)):
      if end_matrix[i]-start_matrix[i]<20:
        del_start_matrix.append(i)
        del_end_matrix.append(i)
    for i in range(len(start_matrix)):
      count=0
      for j in range(len(del_start_matrix)):
        if i==del_start_matrix[j]:
          count=count+1
      if count==0:
        begin_matrix.append(start_matrix[i])
    for i in range(len(end_matrix)):
      count=0
      for j in range(len(del_end_matrix)):
        if i==del_end_matrix[j]:
          count=count+1
      if count==0:
        stop_matrix.append(end_matrix[i])
    
    for i in range(len(begin_matrix)):
      lines.append(img[begin_matrix[i]:stop_matrix[i],:])

      

      

  
  return lines




def word_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a word
  dissection_matrix=[]
  
  words=[]
  m,n=img.shape
  
  length=[]
  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  
  
  start_count=0
  
  
  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]>0 and vertical_hist[0][i-1]==0:
      start_count+=1
      start_matrix.append(i)
    if vertical_hist[0][i]==0 and start_count>0 and vertical_hist[0][i-1]>0:
      end_matrix.append(i)
  
  
  length_mag=0
  for i in range(len(start_matrix)):
    if i>0:
      length_mag=(start_matrix[i]-end_matrix[i-1])
      length.append(length_mag)

  max=np.max(length)
     
  avg=max/3

  dissection_matrix.append([start_matrix[0],end_matrix[0]])
  j=0
  for i in range(len(length)-1):
    
    if length[i]> avg:
      dissection_matrix.append([start_matrix[i+1],end_matrix[i+1]])
      j=j+1
      
    if length[i]<=avg:
      dissection_matrix[j][1]=end_matrix[i+1]
  for i in range(len(dissection_matrix)):
    
    words.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
  
  return words


def char_segmentation(img):
  start_matrix=[]
  dissection_matrix=[]
  delete_matrix=[]
  address_matrix=[]
  
  
  characters=[]
  m,n=img.shape

  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  start_matrix.append(0)

  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]<10:
      start_matrix.append(i)

  for i in range(len(start_matrix)-1):
    if start_matrix[i+1]-start_matrix[i]<10:
      delete_matrix.append(i)
  
  for i in range(len(start_matrix)):
    count=0
    for j in range(len(delete_matrix)):
      if  i==delete_matrix[j]:
        count=count+1
    if count==0:
      address_matrix.append(start_matrix[i])


  for i in range(len(address_matrix)-1):
    dissection_matrix.append([address_matrix[i],address_matrix[i+1]])
  for i in range(len(dissection_matrix)):
    characters.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
    
    
  return characters

In [3]:
directory='/content/data-directory'
def data_label():

  samples=[]
  labels=[]

 
  
 
  
  
  
  for filename in os.listdir(directory):
    count=0
    text=open('/content/lines_new.txt')
    image_orig=Image.open('/content/data-directory/'+filename,'r')
    img=image_binarization(image_orig)
    lines=line_segmentation(img)
    for line in text:
      line_Split = line.strip().split(" ")
      filename_split=filename.split('.')
      linesplit=line_Split[0].split("-")
      string= linesplit[0]+'-'+linesplit[1]
      if string==filename_split[0]:
          count=count+1
    if count==len(lines): 
      for i in range(len(lines)):
        try:
          words=word_segmentation(lines[i])
          text=open('/content/lines_new.txt')
          for line in text:
            lineSplit = line.strip().split(' ')
              
            filename_split=filename.split('.')
            if lineSplit[0]==filename_split[0]+'-'+str(0)+str(i):
              word_split = lineSplit[8].split('|')
              if len(word_split)==len(words):
                for j in range(len(words)):
                  characters=char_segmentation(words[j])
                  if len(word_split[j])==len(characters):
                    for k in range(len(characters)):
                      
                      character=cv2.resize(characters[k],(96,96))
                      samples.append(character)
                    for l in word_split[j]:
                      labels.append(l) 

        except ValueError as ve:
            continue  
          
    
  
                 
  
  return  samples,labels

samples,labels=data_label()
print(len(samples))
print(len(labels))
    

8474
8474


In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
samples = np.array(samples, dtype='float' )/ 255.0
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
samples.shape



(8474, 96, 96)

In [5]:
samples = np.array(samples, dtype='float' )/ 255.0
labels = np.array(labels)
samples.shape

(8474, 96, 96)

In [6]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(samples, labels, test_size=0.2, random_state=42)

In [7]:
trainx=trainx.reshape(trainx.shape[0],96,96,1)
testx=testx.reshape(testx.shape[0],96,96,1)


In [8]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import utils as np_utils
from tensorflow.keras import backend 
img_dims=(96,96,1)
batch_size=64
epoch=50


In [10]:
aug = ImageDataGenerator( rotation_range=25, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')

In [14]:
def build_model(height, width,depth, classes):
  input_shape = (height, width, depth)
  channel_dim = -1 # last position 
  if backend.image_data_format() == 'channels_first':
    input_shape = (depth, height, width)
    channel_dim = 1
    
  model = keras.models.Sequential()
  # BLOCK1
  model.add(Conv2D(32,(3,3),padding='same', input_shape=input_shape))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channel_dim))
  model.add(MaxPooling2D(pool_size=(3, 3)))
  

  model.add(Conv2D(64,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channel_dim))
  
  model.add(Dropout(0.25))

  model.add(Conv2D(128,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channel_dim))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(256,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channel_dim))
  model.add(Dropout(0.5))

  model.add(Conv2D(512,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channel_dim))
  model.add(Dropout(0.5))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(1024,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channel_dim))
  model.add(Dropout(0.5))
  
  
  

  


  
  
    

   
   
   

  

    # BLOCK 5
  model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
  
  model.add(Dense(1024))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=channel_dim))
  model.add(Dropout(0.5))
  
  
  
  

  model.add(Dense(classes))
  model.add(Activation("softmax"))
  
    

    
  return model

trainx=trainx.reshape(trainx.shape[0],96,96,1)
testx=testx.reshape(testx.shape[0],96,96,1)

In [ ]:
epochs_needed=[]
model =build_model(height=img_dims[0], width=img_dims[1],depth=img_dims[2],classes = len(lb.classes_))
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=100, verbose=0, mode='auto', restore_best_weights=True)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])
hist = model.fit(aug.flow(trainx, trainy, batch_size=batch_size),
                 validation_data=(testx, testy),callbacks=[monitor],
                 steps_per_epoch= len(trainx) // batch_size,
                 epochs=100, verbose=1)
epochs = monitor.stopped_epoch
epochs_needed.append(epochs)


In [ ]:
x=testx[1].reshape(1,testx[1].shape[0],testx[1].shape[1],testx[1].shape[2])
a=np.argmax(model.predict(x), axis=-1)
print(lb.classes_[a])

['h']


In [ ]:
image_orig=Image.open('test.png','r')
img=image_binarization(image_orig)
lines=line_segmentation(img)
string=[]
for i in range(len(lines)):
  try:
    words=word_segmentation(lines[i])
    
    for j in range(len(words)):
      characters=char_segmentation(words[j])
      
      for k in range(len(characters)):
        character=cv2.resize(characters[k],(96,96))
        
        
        x=character.reshape(1,96,96,1)
        a=np.argmax(model.predict(x), axis=-1)
        string.append(lb.classes_[a])


  except ValueError as ve:
    continue  




          
